# Use a DNN to learn the mapping between a hex string and a digit string

The goal is to use a Keras Sequential model to learn the mapping between a hex string (e.g. "dbb9f") and digit string (e.g. "899999"). The digit string represents the base numerical 10 value of the hex string.

In [22]:
import os
import keras
from keras import models
from keras import layers
import numpy as np
import pandas as pd

import tensorflow as tf
from keras.layers import Input, Dense, Reshape, Flatten
from keras.models import Sequential
import numpy as np


## Data and Methods for One Hot Encoding

In [23]:
digits = "0123456789"
hex_digits = digits + "abcdef"

digits_char_to_int = dict((c, i) for i, c in enumerate(digits))
digits_int_to_char = dict((i, c) for i, c in enumerate(digits))

hex_char_to_int = dict((c, i) for i, c in enumerate(hex_digits))
hex_int_to_char = dict((i, c) for i, c in enumerate(hex_digits))

# Encoding

def one_hot_encode_hex_string(str):
    int_encoded = [hex_char_to_int[c] for c in str]
    return keras.utils.to_categorical(int_encoded, num_classes=len(hex_digits))

def one_hot_encode_digits_string(str):
    int_encoded = [digits_char_to_int[c] for c in str]
    return keras.utils.to_categorical(int_encoded, num_classes=len(digits))

# Decoding

def one_hot_decode_hex_to_str(arr):
    s = ""
    for row in arr:
        s += hex_int_to_char[np.argmax(row)]
    return s   

def one_hot_decode_digits_to_str(arr):
    s = ""
    for row in arr:
        s += digits_int_to_char[np.argmax(row)]
    return s   

## Example hex string
#### Model Input

In [24]:
# A input/hex string is *always* 5 characters
# Range: "186a0" - "dbb9f"

hex_str = "186a2"

one_hot_encoded = one_hot_encode_hex_string(hex_str)
print(one_hot_encoded)
print(one_hot_encoded.shape)
original_string = one_hot_decode_hex_to_str(one_hot_encoded)
print(original_string)

[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
(5, 16)
186a2


## Example digit string
#### Training label

In [25]:
# A label/digit string is *always* 6 characters
# Range: "100000" - "899999"

digit_str = "100002"

one_hot_encoded = one_hot_encode_digits_string(digit_str)
print(one_hot_encoded)
print(one_hot_encoded.shape)
original_string = one_hot_decode_digits_to_str(one_hot_encoded)
print(original_string)

[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
(6, 10)
100002


## Create Data

In [26]:
def create_csv(fname, start_num, count):
    """
    Create a csv named fname containing n count.
    Each row will contain:
        * number (starting at start_num)
            * incremented by 1 each row
        * the hex value of the string version of number
    """
    with open(fname, "w+") as f:
        result_str = ""

        end_num = start_num + count
        i = start_num

        while i < end_num:
            s = str(i)
            hex_str = hex(i)[2:] # drop the leading "0x"

            result_str += "{},{}\n".format(s, hex_str)
            i += 1

        f.write("string,hex\n")
        f.write(result_str)
        
data_filename = 'data.csv'        

# Create it if it doesn't already exist
if not os.path.isfile(data_filename):
    # from 100_000 (0x186a0) to 899_999 (0xdbb9f)
    create_csv(data_filename, 100000, 800000)

## Load data

In [27]:
df = pd.read_csv(data_filename, dtype={'string': str, 'hex': str})

# Used to test with less data
use_full_dataset = False

if not use_full_dataset:
    df = df[0:5]

# randomize rows when using the full dataset
if use_full_dataset:
    df = df.sample(frac=1).reset_index(drop=True)
    df.set_index('string')

df.head()

,string,hex
0,100000,186a0
1,100001,186a1
2,100002,186a2
3,100003,186a3
4,100004,186a4


## One hot encode all hex and label strings

In [28]:
all_labels_encoded = []
all_hexes_encoded   = []

for row in df.itertuples():
    label_string = row[1] # row['string']
    hex_str      = row[2] # row['hex']

    label_encoded = one_hot_encode_digits_string(label_string)
    hex_encoded   = one_hot_encode_hex_string(hex_str)

    all_labels_encoded.append(label_encoded)
    all_hexes_encoded.append(hex_encoded)

all_labels_encoded = np.asarray(all_labels_encoded)
all_hexes_encoded  = np.asarray(all_hexes_encoded)

print(all_labels_encoded.shape)
print(all_hexes_encoded.shape)

(5, 6, 10)
(5, 5, 16)


## Verify that encoding decoding works

In [29]:
for i in range(5):
    print(one_hot_decode_digits_to_str(all_labels_encoded[i]), 
          "->", 
          one_hot_decode_hex_to_str(all_hexes_encoded[i]))

('100000', '->', '186a0')
('100001', '->', '186a1')
('100002', '->', '186a2')
('100003', '->', '186a3')
('100004', '->', '186a4')


## Split encoded arrays into training, validation, and test sets

In [30]:
if use_full_dataset:
    n_training_examples   = int(example_cnt * 0.7)
    n_validation_examples = int(example_cnt * 0.15)
    n_test_examples       = int(example_cnt * 0.15)
else:
    n_training_examples   = 3
    n_validation_examples = 1
    n_test_examples       = 1

print("Total expected  :", len(df))
print("Total calculated:", n_training_examples + n_validation_examples + n_test_examples)

x_train = all_hexes_encoded[:n_training_examples]
y_train = all_labels_encoded[:n_training_examples]
print("Training examples:", len(x_train))

x_validation = all_hexes_encoded[n_training_examples  : n_training_examples + n_validation_examples]
y_validation = all_labels_encoded[n_training_examples : n_training_examples + n_validation_examples]
print("Validation examples:", len(x_validation))

x_test = all_hexes_encoded[len(all_hexes_encoded) - n_test_examples:]
y_test = all_labels_encoded[len(all_labels_encoded) - n_test_examples:]
print("Test examples", len(x_test))

('Total expected  :', 5)
('Total calculated:', 5)
('Training examples:', 3)
('Validation examples:', 1)
('Test examples', 1)


## Shape of training and label data

In [31]:
print("x_train.shape:", x_train.shape)
print("y_train.shape:", y_train.shape)

('x_train.shape:', (3, 5, 16))
('y_train.shape:', (3, 6, 10))


## Verify our dimensions match our expectations

In [32]:
#
# Dimensions of the input matrix: one hot encoded hex string (.e.g "186a2")
#

# fixed length (in chars) of a hex string
hex_n = len(one_hot_decode_hex_to_str(all_hexes_encoded[0]))

# number of symbols in the "alphabet" of a hex string
hex_k = len(hex_digits)

print("Training dimensions match:", x_train.shape[1:3] == (hex_n, hex_k))

#
# Dimensions of the label matrix: one hot encoded digits string (.e.g "100002")
#

# fixed length (in chars) of a hex string
label_n = len(one_hot_decode_digits_to_str(all_labels_encoded[0]))

# number of symbols in the "alphabet" of a digits string
label_k = len(digits)

print("Label dimensions match:", y_train.shape[1:3] == (label_n, label_k))

('Training dimensions match:', True)
('Label dimensions match:', True)


# Build and compile model

### Questions
* How do we determine the correct number of nodes in the initial layer?
* The shape of the input data is `(5, 16)` and the shape of our labels is `(6, 10)`. How do we define a model that can convert between these differing dimensions?
* Do we have to create our own [loss function implementation](https://stackoverflow.com/questions/43576922/keras-custom-metric-iteration) to make this work?

In [65]:
n = 5 #input size
m = 6 #output size
l = 16 #input alphabet size
k = 10 #output alpahbet size
bs = 2 #batch size


model = Sequential()
model.add(Flatten(input_shape = (n,l)))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(m * k, activation=None))
model.add(Reshape((m, k)))

def batch_accuracy(truth, pred):
    truth_decoded = tf.argmax(truth, axis = 2)
    pred_decoded = tf.argmax(pred, axis = 2)
    correct = tf.cast(tf.equal(truth, pred), tf.float32)
    #correct is bs x m
    accuracy = tf.reduce_sum(correct, axis = 1)/float(m)
    return accuracy

def batch_crossentropy(truth, pred):
    batch_size = tf.shape(truth)[0]
    #input shapes are BxMxK where B is batchsize
    #reshape so that every row is a separate probability distribution
    truth = tf.reshape(truth, (-1, k))
    pred = tf.reshape(pred, (-1, k))
    ce = tf.nn.softmax_cross_entropy_with_logits(labels = truth, logits = pred)
    #crossentropy is now bs * m, and we want it reshaped to bs x m
    ce = tf.reshape(ce, (batch_size,-1))
    loss = tf.reduce_sum(ce, axis = 1)
    return loss

model.compile(optimizer='rmsprop', 
              loss=batch_crossentropy,
              metrics = [batch_accuracy])


## Train

In [66]:
model.fit(x_train, y_train, 
          batch_size = bs, verbose = 1, epochs = 200, 
          validation_data=(x_validation, y_validation))

Train on 3 samples, validate on 1 samples
Epoch 1/200
3/3 [==============================] - 0s - loss: 13.6475 - batch_accuracy: 0.0000e+00 - val_loss: 13.0961 - val_batch_accuracy: 0.0000e+00
Epoch 2/200
3/3 [==============================] - 0s - loss: 13.0114 - batch_accuracy: 0.0000e+00 - val_loss: 12.6048 - val_batch_accuracy: 0.0000e+00
Epoch 3/200
3/3 [==============================] - 0s - loss: 12.4358 - batch_accuracy: 0.0000e+00 - val_loss: 12.0501 - val_batch_accuracy: 0.0000e+00
Epoch 4/200
3/3 [==============================] - 0s - loss: 11.7728 - batch_accuracy: 0.0000e+00 - val_loss: 11.3880 - val_batch_accuracy: 0.0000e+00
Epoch 5/200
3/3 [==============================] - 0s - loss: 10.9984 - batch_accuracy: 0.0000e+00 - val_loss: 10.6268 - val_batch_accuracy: 0.0000e+00
Epoch 6/200
3/3 [==============================] - 0s - loss: 10.1174 - batch_accuracy: 0.0000e+00 - val_loss: 9.8076 - val_batch_accuracy: 0.0000e+00
Epoch 7/200
3/3 [==============================

3/3 [==============================] - 0s - loss: 0.9349 - batch_accuracy: 0.0000e+00 - val_loss: 6.6105 - val_batch_accuracy: 0.0000e+00
Epoch 54/200
3/3 [==============================] - 0s - loss: 0.9391 - batch_accuracy: 0.0000e+00 - val_loss: 6.7024 - val_batch_accuracy: 0.0000e+00
Epoch 55/200
3/3 [==============================] - ETA: 0s - loss: 0.7953 - batch_accuracy: 0.0000e+ - 0s - loss: 0.9406 - batch_accuracy: 0.0000e+00 - val_loss: 6.7557 - val_batch_accuracy: 0.0000e+00
Epoch 56/200
3/3 [==============================] - 0s - loss: 0.9211 - batch_accuracy: 0.0000e+00 - val_loss: 6.8160 - val_batch_accuracy: 0.0000e+00
Epoch 57/200
3/3 [==============================] - 0s - loss: 0.9217 - batch_accuracy: 0.0000e+00 - val_loss: 6.9065 - val_batch_accuracy: 0.0000e+00
Epoch 58/200
3/3 [==============================] - 0s - loss: 0.9206 - batch_accuracy: 0.0000e+00 - val_loss: 6.9683 - val_batch_accuracy: 0.0000e+00
Epoch 59/200
3/3 [==============================] - 0s 

3/3 [==============================] - 0s - loss: 0.4160 - batch_accuracy: 0.0000e+00 - val_loss: 10.6343 - val_batch_accuracy: 0.0000e+00
Epoch 106/200
3/3 [==============================] - ETA: 0s - loss: 0.3870 - batch_accuracy: 0.0000e+ - 0s - loss: 0.4127 - batch_accuracy: 0.0000e+00 - val_loss: 10.7375 - val_batch_accuracy: 0.0000e+00
Epoch 107/200
3/3 [==============================] - 0s - loss: 0.4084 - batch_accuracy: 0.0000e+00 - val_loss: 10.7500 - val_batch_accuracy: 0.0000e+00
Epoch 108/200
3/3 [==============================] - ETA: 0s - loss: 0.3224 - batch_accuracy: 0.0000e+ - 0s - loss: 0.3810 - batch_accuracy: 0.0000e+00 - val_loss: 10.8124 - val_batch_accuracy: 0.0000e+00
Epoch 109/200
3/3 [==============================] - 0s - loss: 0.3777 - batch_accuracy: 0.0000e+00 - val_loss: 10.8883 - val_batch_accuracy: 0.0000e+00
Epoch 110/200
3/3 [==============================] - 0s - loss: 0.3641 - batch_accuracy: 0.0000e+00 - val_loss: 10.9666 - val_batch_accuracy: 0.0

3/3 [==============================] - 0s - loss: 0.0446 - batch_accuracy: 0.0000e+00 - val_loss: 14.2396 - val_batch_accuracy: 0.0000e+00
Epoch 158/200
3/3 [==============================] - 0s - loss: 0.0441 - batch_accuracy: 0.0000e+00 - val_loss: 14.4772 - val_batch_accuracy: 0.0000e+00
Epoch 159/200
3/3 [==============================] - 0s - loss: 0.0360 - batch_accuracy: 0.0000e+00 - val_loss: 14.4136 - val_batch_accuracy: 0.0000e+00
Epoch 160/200
3/3 [==============================] - ETA: 0s - loss: 0.0440 - batch_accuracy: 0.0000e+ - 0s - loss: 0.0328 - batch_accuracy: 0.0000e+00 - val_loss: 14.4265 - val_batch_accuracy: 0.0000e+00
Epoch 161/200
3/3 [==============================] - 0s - loss: 0.0313 - batch_accuracy: 0.0000e+00 - val_loss: 14.4926 - val_batch_accuracy: 0.0000e+00
Epoch 162/200
3/3 [==============================] - 0s - loss: 0.0296 - batch_accuracy: 0.0000e+00 - val_loss: 14.5436 - val_batch_accuracy: 0.0000e+00
Epoch 163/200
3/3 [=========================

# Evaluate the trained model on data it has never seen

In [62]:
if False:
    results = model.evaluate(x_test, y_test)
    print("\nloss    : {}\naccuracy: {}".format(results[0], results[1] * 100))
    
output = model.predict(x_test)[0]
pred_digits = one_hot_decode_digits_to_str(output)
truth_digits = one_hot_decode_digits_to_str(y_test[0])

print 'predicted :', pred_digits
print 'truth :', truth_digits

predicted : 100001
truth : 100004
